In [11]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
telecom_df = pd.read_csv('filtered.csv')
telecom_df.drop(['Unnamed: 0','mobile_number'],axis=1,inplace=True)
telecom_df = telecom_df[~(telecom_df['dataHeavy_goodphase'].isnull() | telecom_df['dataHeavy_8'].isnull())]

In [18]:
#Test-Train
from sklearn.model_selection import train_test_split
X = telecom_df.drop('churn',axis=1)
y = telecom_df['churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, random_state = 99)
X_train.head()

,arpu_8,onnet_mou_8,offnet_mou_8,roam_ic_mou_8,roam_og_mou_8,loc_og_t2t_mou_8,loc_og_t2m_mou_8,loc_og_t2f_mou_8,loc_og_t2c_mou_8,loc_og_mou_8,std_og_t2t_mou_8,std_og_t2m_mou_8,std_og_t2f_mou_8,std_og_mou_8,isd_og_mou_8,spl_og_mou_8,og_others_8,total_og_mou_8,loc_ic_t2t_mou_8,loc_ic_t2m_mou_8,loc_ic_t2f_mou_8,loc_ic_mou_8,std_ic_t2t_mou_8,std_ic_t2m_mou_8,std_ic_t2f_mou_8,std_ic_mou_8,total_ic_mou_8,spl_ic_mou_8,isd_ic_mou_8,ic_others_8,total_rech_num_8,total_rech_amt_8,max_rech_amt_8,date_of_last_rech_8,last_day_rch_amt_8,date_of_last_rech_data_8,total_rech_data_8,max_rech_data_8,count_rech_2g_8,count_rech_3g_8,av_rech_amt_data_8,vol_2g_mb_8,vol_3g_mb_8,arpu_3g_8,arpu_2g_8,night_pck_user_8,monthly_2g_8,sachet_2g_8,monthly_3g_8,sachet_3g_8,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,av_rech_67,recency_goodphase,recency_data_goodphase,data_volm_used_per_rupee_8,dataHeavy_8,loc_og_t2t_mou_goodphase,loc_ic_t2m_mou_goodphase,loc_ic_mou_goodphase,total_rech_num_goodphase,onnet_mou_goodphase,count_rech_3g_goodphase,vol_3g_mb_goodphase,roam_ic_mou_goodphase,loc_og_t2m_mou_goodphase,total_rech_amt_goodphase,loc_og_t2f_mou_goodphase,std_og_t2m_mou_goodphase,loc_og_t2c_mou_goodphase,std_ic_t2f_mou_goodphase,loc_ic_t2t_mou_goodphase,total_rech_data_goodphase,arpu_2g_goodphase,count_rech_2g_goodphase,roam_og_mou_goodphase,og_others_goodphase,loc_og_mou_goodphase,std_og_mou_goodphase,ic_others_goodphase,total_og_mou_goodphase,std_og_t2t_mou_goodphase,arpu_3g_goodphase,isd_og_mou_goodphase,vol_2g_mb_goodphase,isd_ic_mou_goodphase,std_ic_mou_goodphase,std_ic_t2m_mou_goodphase,total_ic_mou_goodphase,spl_ic_mou_goodphase,spl_og_mou_goodphase,std_ic_t2t_mou_goodphase,loc_ic_t2f_mou_goodphase,offnet_mou_goodphase,std_og_t2f_mou_goodphase,arpu_goodphase,av_rech_amt_data_goodphase,data_volm_used_per_rupee_goodphase,fb_user_goodphase,night_pck_user_goodphase,sachet_2g_goodphase,sachet_3g_goodphase,monthly_2g_goodphase,monthly_3g_goodphase,dataHeavy_goodphase,last_day_rch_amt_goodphase,max_rech_data_goodphase,max_rech_amt_goodphase
5406,899.485,178.61,1544.74,9.05,57.65,81.26,339.71,0.0,0.00,420.98,96.99,1147.73,0.0,1244.73,0.0,0.00,0.0,1665.71,26.93,294.78,4.78,326.49,25.21,19.96,0.0,45.18,422.76,0.00,44.86,6.21,8,899,154,27.0,154,27.0,2.0,154.0,2.0,0.0,179.0,587.23,0.00,0.00,0.01,0.0,1,1,0,0,1.0,1013,0.00,0.00,0.00,0.0,111.586207,1.0,1.0,3.280615,0.0,52.174,189.752,219.469,14.7,84.941,0.0,0.000,21.056,359.634,1694.8,0.882,1430.560,0.0,0.0,24.404,1.7,166.780,1.7,127.666,0.000,412.700,1454.766,0.000,1871.151,24.196,0.000,0.000,319.566,10.631,24.554,23.183,254.664,0.000,3.675,1.368,5.296,1913.873,0.0,1326.6532,292.6,1.260272,1.0,0.0,0,0,2,0,0.0,198,198.0,229
28386,1745.936,0.00,0.73,0.00,0.00,0.00,0.73,0.0,0.31,0.73,0.00,0.00,0.0,0.00,0.0,2.76,0.0,3.49,0.00,2.23,0.00,2.23,0.00,0.00,0.0,0.00,2.33,0.00,0.00,0.10,6,2203,649,26.0,649,21.0,2.0,252.0,0.0,2.0,504.0,64.41,1077.50,426.29,424.34,0.0,0,0,2,0,1.0,1122,639.22,303.29,427.54,0.0,222.250000,1.0,24.0,2.265694,0.0,4.655,15.126,15.315,7.6,4.669,1.3,956.964,0.000,20.240,1707.6,0.000,12.366,0.0,0.0,0.189,1.3,275.821,0.0,0.000,0.714,24.905,12.380,0.000,44.181,0.007,275.824,2.031,15.953,0.000,0.000,0.000,15.315,0.000,4.131,0.000,0.000,34.570,0.0,1543.9559,327.6,2.889419,1.0,0.0,0,0,0,1,0.0,250,252.0,550
27498,223.381,15.59,61.93,0.00,0.00,15.59,58.68,0.0,0.00,74.28,0.00,1.01,0.0,1.01,0.0,2.23,0.0,77.53,96.38,169.69,4.95,271.03,3.53,11.14,0.0,14.68,288.03,0.00,2.31,0.00,3,204,154,25.0,50,5.0,1.0,154.0,1.0,0.0,154.0,440.32,0.00,0.00,6.81,0.0,1,0,0,0,1.0,1958,0.00,0.00,0.00,0.0,66.000000,12.0,23.0,2.859221,0.0,15.854,97.470,206.730,3.6,19.455,0.0,0.000,0.000,58.020,268.0,1.075,0.000,0.0,0.0,84.280,1.0,-0.020,1.0,0.000,0.000,74.959,3.598,0.336,80.489,3.598,0.000,0.000,490.870,0.175,12.556,10.519,219.831,0.024,1.929,2.037,24.970,61.027,0.0,204.1360,154.0,3.187468,1.0,0.0,0,0,1,0,0.0,120,154.0,154
22991,128.728,0.00,0.23,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.23,0.0,0.23,0.00,0.0

In [34]:
df = pd.DataFrame(X_train_pca,columns=X_train.columns)

In [38]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

folds = KFold(n_splits = 5, shuffle = True, random_state = 4)
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}]
model = SVC(kernel="rbf")
model_cv = GridSearchCV(estimator = model, param_grid = hyper_params, scoring= 'accuracy', 
                        cv = folds, verbose = 1,return_train_score=True)      
model_cv.fit(X_train, Y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
best_params = {"C": 100, "gamma": 0.0001, "kernel":"rbf"}
model = SVC(C=100, gamma=0.0001, kernel="rbf")
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(metrics.confusion_matrix(Y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(Y_test, y_pred))
print("precision", metrics.precision_score(Y_test, y_pred))
print("sensitivity/recall", metrics.recall_score(Y_test, y_pred))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(svd_solver='randomized', random_state=42,n_components=0.95, whiten=True)
pc = pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_train_pca = pd.DataFrame(X_train_pca,columns=X_train.columns)
X_test_pca = pd.DataFrame(X_test_pca,columns=X_train.columns)

In [ ]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 4)
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}]
model = SVC(kernel="rbf")
model_cv = GridSearchCV(estimator = model, param_grid = hyper_params, scoring= 'accuracy', 
                        cv = folds, verbose = 1,return_train_score=True)      
model_cv.fit(X_pca_train, Y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
best_params = {"C": 100, "gamma": 0.0001, "kernel":"rbf"}
model = SVC(C=100, gamma=0.0001, kernel="rbf")
model.fit(X_pca_train, Y_train)
y_pred = model.predict(X_pca_test)
print(metrics.confusion_matrix(Y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(Y_test, y_pred))
print("precision", metrics.precision_score(Y_test, y_pred))
print("sensitivity/recall", metrics.recall_score(Y_test, y_pred))